## M1 - Model prediksi titik panas di Sumatera Selatan 2001 - 2023

In [ ]:
# load all functions
from C01_data_collection import *
from C02_visualization import *
from C03_preprocessing import *
from C04_model_predictions import *
# from C05_model_evaluate import *

- config models

In [ ]:
# set random number
import random as rm
rm.seed(1234)

# set random number
import numpy as np
np.random.seed(1234)

# set random number
import tensorflow as tf
tf.random.set_seed(1234)

### 1. Akuisisi Data

In [ ]:
# load dataset
dataset = data_hotspot("dataset_enso.csv")
print(dataset.tail())

In [ ]:
# hasil load dataset
lineplot1(x=dataset["acq_date"], y=dataset["hotspot"], label="hotspot", title="Hotspots in South Sumatra 2001 - 2023")

### 2. Praproses Data

In [ ]:
# proses normalisasi min-max
scaler, scaled = normalized(dataset, ["hotspot"])

In [ ]:
# hasil normalisasi min-max
lineplot1(x=dataset["acq_date"], y=scaled, label="hotspot", title="Hotspots in South Sumatra 2001 - 2023")

### 3. Pembagian Data

In [ ]:
# proses pembagian data
train_data, test_data = splitting(scaled)
print(train_data.shape, test_data.shape)

In [ ]:
# hasil pembagian data
lineplot2(
  x1=dataset["acq_date"].iloc[:len(train_data)], y1= train_data, label1="Training Data",
  x2=dataset["acq_date"].iloc[len(train_data):], y2=test_data, label2="Testing Data",
  title="Results of Splitting Data"
)

### 4. Supervised Learning

In [ ]:
# hasil supervised learning
x_train, y_train, x_test, y_test = results_univariate_supervised(train_data, test_data)

In [ ]:
# cek dimensi data
print(x_train.shape, y_train.shape)

In [ ]:
# cek dimensi data
print(x_test.shape, y_test.shape)

### 5.1 Modeling SBi-LSTM

- SBi-LSTM

In [ ]:
# call models LSTM
# param : algorithm, timestep, activation, optimizer, dropout
lstm_model = get_models(
  algorithm="SBi-LSTM", timestep=x_train, activation="selu", optimizer="sgd", dropout=0.25
)

In [ ]:
# compile models
# param : def model, batch_size, epochs, x_train, y_train, x_test, y_test
lstm_history, lstm_predictions = get_prediction(
  model=lstm_model, batch_size=8, epochs=1500, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test
)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(
  y_test, lstm_predictions
)

In [ ]:
# hasil pembagian data
lineplot2(
  x1=dataset["acq_date"].iloc[len(train_data)+1:], y1= y_test, label1="Training Data",
  x2=dataset["acq_date"].iloc[len(train_data)+1:], y2=lstm_predictions, label2="Testing Data",
  title="Results of Splitting Data"
)

In [ ]:
inv_x_train = scaler.inverse_transform(x_train.reshape(-1,1))
inv_y_train = scaler.inverse_transform(y_train.reshape(-1,1))
inv_x_test  = scaler.inverse_transform(x_test.reshape(-1,1))
inv_y_test  = scaler.inverse_transform(y_test.reshape(-1,1))

In [ ]:
inv_lstm_predictions = scaler.inverse_transform(lstm_predictions.reshape(-1,1))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(
  y_test, lstm_predictions
)